# Compile the results on the MOF case study 

In [36]:
from glob import glob
from pathlib import Path
import matplotlib.pyplot as plt
plt.style.use(['science', 'nature'])
import pandas as pd 
from fastcore.helpers import load_pickle
from gpt3forchem.helpers import get_else_nan

## QMOF 

### Classification 

In [37]:
all_qmof_classification_res = glob('/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/results/20220913_mof_classification/*.pkl')

In [38]:
compiled_qmof_classification_data = []
compiled_qmof_baseline_data = []

for filename in all_qmof_classification_res:
    loaded_res = load_pickle(filename)

    res = {
        "accuracy": get_else_nan(loaded_res["cm"], "ACC_Macro"),
        "f1_macro": get_else_nan(loaded_res["cm"], "F1_Macro"),
        "f1_micro": get_else_nan(loaded_res["cm"], "F1_Micro"),
        "train_size": loaded_res["train_size"],
        "mcc": get_else_nan(loaded_res["cm"], "Overall_MCC"),
        "representation": loaded_res["representation"],
        "target": loaded_res["target"],
    }

    baseline_res = {
        "baseline_accuracy": get_else_nan(loaded_res["baseline_cm"], "ACC_Macro"),
        "baseline_f1_macro": get_else_nan(loaded_res["baseline_cm"], "F1_Macro"),
        "baseline_f1_micro": get_else_nan(loaded_res["baseline_cm"], "F1_Micro"),
        "baseline_mcc": get_else_nan(loaded_res["baseline_cm"], "Overall_MCC"),
        "train_size": loaded_res["train_size"],
        "representation": loaded_res["representation"],
        "target": loaded_res["target"],
    }

    compiled_qmof_classification_data.append(res)
    compiled_qmof_baseline_data.append(baseline_res)


In [39]:
load_pickle(all_qmof_classification_res[0])

{'model_type': 'ada',
 'train_set_size': 50,
 'prefix': '',
 'train_size': 50,
 'test_size': 3577,
 'cm': pycm.ConfusionMatrix(classes: ['0', '1', '2', '3', '4']),
 'accuracy': 0.76,
 'completions': {'choices': [<OpenAIObject at 0x16abd8680> JSON: {
     "finish_reason": "length",
     "index": 0,
     "logprobs": null,
     "text": " 2@@@@@@@@@@@@@@"
   },
   <OpenAIObject at 0x16abd83b0> JSON: {
     "finish_reason": "length",
     "index": 1,
     "logprobs": null,
     "text": " 2@@@@@@@@@@@@@@"
   },
   <OpenAIObject at 0x16abd8540> JSON: {
     "finish_reason": "length",
     "index": 2,
     "logprobs": null,
     "text": " 2@@@@@@@@@@@@@@"
   },
   <OpenAIObject at 0x16abd8720> JSON: {
     "finish_reason": "length",
     "index": 3,
     "logprobs": null,
     "text": " 2@@@@@@@@@@@@@@"
   },
   <OpenAIObject at 0x16abd87c0> JSON: {
     "finish_reason": "length",
     "index": 4,
     "logprobs": null,
     "text": " 2@@@@@@@@@@@@@@"
   },
   <OpenAIObject at 0x16abd8950> JSO

In [40]:
compiled_qmof_classification_data = pd.DataFrame(compiled_qmof_classification_data)
compiled_qmof_baseline_data = pd.DataFrame(compiled_qmof_baseline_data)

In [41]:
compiled_qmof_classification_data.groupby(['target', 'representation', 'train_size']).agg(['mean', 'std'])

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_34742/4159097433.py:1: FutureWarning: ['mcc'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  compiled_qmof_classification_data.groupby(['target', 'representation', 'train_size']).agg(['mean', 'std'])


accuracy            \
                                                               mean       std   
target                  representation         train_size                       
outputs.pbe.bandgap_cat chemical_name          10          0.735467  0.021914   
                                               50          0.768800  0.041899   
                                               100         0.818133  0.014048   
                                               200         0.836400  0.003960   
                                               500         0.855200       NaN   
                                               1000        0.879533  0.018573   
                                               2000        0.889200  0.003960   
                                               3000        0.922400       NaN   
                        info.mofid.mofid_clean 10          0.717433  0.037845   
                                               50          0.763985  0.024521   
                                               100         0.845259  0.058360   
                                               200         0.830480  0.046759   
                                               500         0.870667  0.007668   
                                               1000        0.883387  0.007723   
                                               2000        0.900533  0.009272   
                                               3000        0.908000  0.015839   

                                                           f1_macro            \
                                                               mean       std   
target                  representation         train_size                       
outputs.pbe.bandgap_cat chemical_name          10          0.194741  0.038253   
                                               50          0.182454  0.098772   
                                               100         0.292088  0.052740   
                                               200         0.360514  0.014498   
                                               500         0.462298       NaN   
                                               1000        0.469777  0.040520   
                                               2000        0.608783  0.064483   
                                               3000        0.621715       NaN   
                        info.mofid.mofid_clean 10          0.141765  0.090571   
                                               50          0.174914  0.070117   
                                               100         0.293975  0.158088   
                                               200         0.368876  0.096230   
                                               500         0.501428  0.014631   
                                               1000        0.512295  0.044672   
                                               2000        0.577524  0.123864   
                                               3000        0.605338  0.035133   

                                                           f1_micro            
                                                               mean       std  
target                  representation         train_size                      
outputs.pbe.bandgap_cat chemical_name          10          0.338667  0.054784  
                                               50          0.422000  0.104747  
                                               100         0.545333  0.035119  
                                               200         0.591000  0.009899  
                                               500         0.638000       NaN  
                                               1000        0.672000  0.008485  
                                               2000        0.723000  0.009899  
                                               3000        0.806000       NaN  
                        info.mofid.mofid_clean 10          0.252500  0.168312  
                   

In [48]:
compiled_qmof_baseline_data

,baseline_accuracy,baseline_f1_macro,baseline_f1_micro,baseline_mcc,train_size,representation,target
0,NaN,NaN,NaN,NaN,50,chemical_name,outputs.pbe.bandgap_cat
1,NaN,NaN,NaN,NaN,10,chemical_name,outputs.pbe.bandgap_cat
2,NaN,NaN,NaN,NaN,50,info.mofid.mofid_clean,outputs.pbe.bandgap_cat
3,NaN,NaN,NaN,NaN,50,info.mofid.mofid_clean,outputs.pbe.bandgap_cat
4,NaN,NaN,NaN,NaN,100,chemical_name,outputs.pbe.bandgap_cat
5,NaN,NaN,NaN,NaN,1000,info.mofid.mofid_clean,outputs.pbe.bandgap_cat
6,NaN,NaN,NaN,NaN,1000,info.mofid.mofid_clean,outputs.pbe.bandgap_cat
7,NaN,NaN,NaN,NaN,500,info.mofid.mofid_clean,outputs.pbe.bandgap_cat
8,NaN,NaN,NaN,NaN,50,info.mofid.mofid_clean,outputs.pbe.bandgap_cat
9,NaN,NaN,NaN,NaN,100,info.mofid.mofid_clean,outputs.pbe.bandgap_cat


### Regression 

In [42]:
qmof_regression_results = glob('/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/results/20220915_mof_regression/*.pkl')

In [43]:
load_pickle(qmof_regression_results[0])

{'model_type': 'ada',
 'train_set_size': 2000,
 'prefix': '',
 'train_size': 2000,
 'test_size': 1627,
 'metrics': {'r2': 0.7192669331671564,
  'max_error': 2.5827999999999998,
  'mean_absolute_error': 0.435709634,
  'mean_squared_error': 0.374205485176178},
 'completions': {'choices': [<OpenAIObject at 0x16aaae450> JSON: {
     "finish_reason": "length",
     "index": 0,
     "logprobs": null,
     "text": " 0.9595@@@4.936"
   },
   <OpenAIObject at 0x16aaaeef0> JSON: {
     "finish_reason": "length",
     "index": 1,
     "logprobs": null,
     "text": " 1.934@@@+1.9"
   },
   <OpenAIObject at 0x16aaae860> JSON: {
     "finish_reason": "length",
     "index": 2,
     "logprobs": null,
     "text": " 0.49@@@@@@ 0.49"
   },
   <OpenAIObject at 0x16aaaed10> JSON: {
     "finish_reason": "length",
     "index": 3,
     "logprobs": null,
     "text": " 0.03934@@@0.49"
   },
   <OpenAIObject at 0x16abee6d0> JSON: {
     "finish_reason": "length",
     "index": 4,
     "logprobs": null,
   

In [44]:
compiled_regression_results = []
compiled_regression_baseline_results = []

for res_file in qmof_regression_results:
    res = load_pickle(res_file)
    res_dict = {
        'r2': res['metrics']['r2'],
        'max_error': res['metrics']['max_error'],
        'mean_absolute_error': res['metrics']['mean_absolute_error'],
        'mean_squared_error': res['metrics']['mean_squared_error'],
    }
    res_dict['train_size'] = res['train_size']
    res_dict['representation'] = res['representation']
    res_dict['target'] = res['target']
    baseline_res_dict = res['baseline_metrics']
    baseline_res_dict['train_size'] = res['train_size']
    baseline_res_dict['target'] = res['target']

    compiled_regression_results.append(res_dict)
    compiled_regression_baseline_results.append(baseline_res_dict)
    

In [45]:
compiled_regression_results = pd.DataFrame(compiled_regression_results)
compiled_regression_baseline_results = pd.DataFrame(compiled_regression_baseline_results)

In [46]:
compiled_regression_results.groupby(['target', 'representation', 'train_size']).agg(['mean', 'std'])

r2            \
                                                           mean       std   
target              representation         train_size                       
outputs.pbe.bandgap info.mofid.mofid_clean 10         -0.386953  0.017388   
                                           50         -0.263669  0.294140   
                                           100        -0.277853  0.424610   
                                           200         0.391439  0.099010   
                                           500         0.578588  0.037843   
                                           1000        0.543437  0.096453   
                                           2000        0.630824  0.089733   
                                           3000        0.699638  0.020831   

                                                      max_error            \
                                                           mean       std   
target              representation         train_size                       
outputs.pbe.bandgap info.mofid.mofid_clean 10          3.491233  0.229320   
                                           50          3.606416  0.658445   
                                           100         3.739493  0.473727   
                                           200         3.713433  0.544137   
                                           500         3.215900  0.710886   
                                           1000        3.088475  0.020962   
                                           2000        2.794920  0.188066   
                                           3000        2.943025  0.208280   

                                                      mean_absolute_error  \
                                                                     mean   
target              representation         train_size                       
outputs.pbe.bandgap info.mofid.mofid_clean 10                    1.110821   
                                           50                    1.031600   
                                           100                   1.014711   
                                           200                   0.690291   
                                           500                   0.555026   
                                           1000                  0.584356   
                                           2000                  0.511169   
                                           3000                  0.467493   

                                                                 \
                                                            std   
target              representation         train_size             
outputs.pbe.bandgap info.mofid.mofid_clean 10          0.028184   
                                           50          0.136183   
                                           100         0.191929   
                                           200         0.052820   
                                           500         0.039834   
                                           1000        0.061217   
                                           2000        0.070023   
                                           3000        0.013075   

                                                      mean_squared_error  \
                                                                    mean   
target              representation         train_size                      
outputs.pbe.bandgap info.mofid.mofid_clean 10                   1.898458   
                                           50                   1.698235   
                                           100                  1.792388   
                                           200                  0.859579   
                                           500                  0.563539   
                                           1000                 0.620125   
                                           2000                 0.496791   
                   

In [49]:
compiled_regression_baseline_results.groupby(['target', 'train_size']).agg(['mean', 'std'])

r2     max_error     mean_absolute_error      \
                               mean std      mean std                mean std   
target              train_size                                                  
outputs.pbe.bandgap 10          NaN NaN       NaN NaN                 NaN NaN   
                    50          NaN NaN       NaN NaN                 NaN NaN   
                    100         NaN NaN       NaN NaN                 NaN NaN   
                    200         NaN NaN       NaN NaN                 NaN NaN   
                    500         NaN NaN       NaN NaN                 NaN NaN   
                    1000        NaN NaN       NaN NaN                 NaN NaN   
                    2000        NaN NaN       NaN NaN                 NaN NaN   
                    3000        NaN NaN       NaN NaN                 NaN NaN   

                               mean_squared_error      
                                             mean std  
target              train_size                         
outputs.pbe.bandgap 10                        NaN NaN  
                    50                        NaN NaN  
                    100                       NaN NaN  
                    200                       NaN NaN  
                    500                       NaN NaN  
                    1000                      NaN NaN  
                    2000                      NaN NaN  
                    3000                      NaN NaN

## CoRE MOF

In [50]:
all_core_results = glob('results/20220914_mof_classification/*pkl')

In [52]:
compiled_core_classification_data = []
compiled_core_baseline_data = []

for filename in all_core_results:
    loaded_res = load_pickle(filename)

    res = {
        "accuracy": get_else_nan(loaded_res["cm"], "ACC_Macro"),
        "f1_macro": get_else_nan(loaded_res["cm"], "F1_Macro"),
        "f1_micro": get_else_nan(loaded_res["cm"], "F1_Micro"),
        "train_size": loaded_res["train_size"],
        "mcc": get_else_nan(loaded_res["cm"], "Overall_MCC"),
        "representation": loaded_res["representation"],
        "target": loaded_res["target"],
    }

    baseline_res = {
        "baseline_accuracy": get_else_nan(loaded_res["baseline_cm"], "ACC_Macro"),
        "baseline_f1_macro": get_else_nan(loaded_res["baseline_cm"], "F1_Macro"),
        "baseline_f1_micro": get_else_nan(loaded_res["baseline_cm"], "F1_Micro"),
        "baseline_mcc": get_else_nan(loaded_res["baseline_cm"], "Overall_MCC"),
        "train_size": loaded_res["train_size"],
        "representation": loaded_res["representation"],
        "target": loaded_res["target"],
    }

    compiled_core_classification_data.append(res)
    compiled_core_baseline_data.append(baseline_res)


In [53]:
compiled_core_classification_data = pd.DataFrame(compiled_core_classification_data)

In [57]:
compiled_core_classification_data_agg = compiled_core_classification_data.groupby(['target', 'representation', 'train_size']).agg(['mean', 'std'])

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_34742/2870106362.py:1: FutureWarning: ['mcc'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  compiled_core_classification_data_agg = compiled_core_classification_data.groupby(['target', 'representation', 'train_size']).agg(['mean', 'std'])


In [58]:
compiled_core_classification_data_agg

accuracy            \
                                                      mean       std   
target                representation  train_size                       
outputs.CH4DC_cat     chemical_name_y 10          0.717800  0.010701   
                                      50          0.727200  0.005581   
                                      100         0.721400  0.006000   
                                      200         0.806800  0.124025   
                                      500         0.735911  0.006534   
                                      1000        0.773067  0.008992   
                      clean_mofid     10          0.745229  0.043210   
                                      50          0.720267  0.010920   
                                      100         0.719733  0.002013   
                                      200         0.736400  0.014816   
                                      500         0.773462  0.059603   
                                      1000        0.756800  0.006882   
outputs.logKH_CO2_cat chemical_name_y 10          0.778433  0.006467   
                                      50          0.771600  0.006450   
                                      100         0.768800  0.012445   
                                      200         0.777000  0.007016   
                                      500         0.789833  0.025002   
                                      1000        0.837956  0.016460   
                      clean_mofid     10          0.764833  0.036884   
                                      50          0.772000  0.007811   
                                      100         0.766133  0.009544   
                                      200         0.782000  0.011323   
                                      500         0.790200  0.005362   
                                      1000        0.803400  0.010681   

                                                  f1_macro            \
                                                      mean       std   
target                representation  train_size                       
outputs.CH4DC_cat     chemical_name_y 10          0.149823  0.027937   
                                      50          0.130674  0.023026   
                                      100         0.153043  0.016027   
                                      200         0.132933  0.123551   
                                      500         0.229552  0.079504   
                                      1000        0.357930  0.034869   
                      clean_mofid     10          0.124279  0.026044   
                                      50          0.160490  0.057974   
                                      100         0.135585  0.020836   
                                      200         0.198593  0.056259   
                                      500         0.210406  0.108564   
                                      1000        0.310452  0.026149   
outputs.logKH_CO2_cat chemical_name_y 10          0.131826  0.009319   
                                      50          0.125988  0.010818   
                                      100         0.156510  0.024739   
                                      200         0.199429  0.050453   
                                      500         0.241819  0.183007   
                                      1000        0.327946  0.103090   
                      clean_mofid     10          0.163316  0.027357   
                                      50          0.168882  0.034863   
                                      100         0.158907  0.029247   
                                      200         0.208806  0.030507   
                                      500         0.224695  0.033614   
                                      1000        0.303062  0.038744   

                                                  f1_micro            
                                                      mean       std  
target            

In [55]:
compiled_core_baseline_data = pd.DataFrame(compiled_core_baseline_data)

In [56]:
compiled_core_baseline_data

,baseline_accuracy,baseline_f1_macro,baseline_f1_micro,baseline_mcc,train_size,representation,target
0,NaN,NaN,NaN,NaN,10,chemical_name_y,outputs.logKH_CO2_cat
1,NaN,NaN,NaN,NaN,200,chemical_name_y,outputs.CH4DC_cat
2,NaN,NaN,NaN,NaN,500,clean_mofid,outputs.CH4DC_cat
3,NaN,NaN,NaN,NaN,10,clean_mofid,outputs.logKH_CO2_cat
4,NaN,NaN,NaN,NaN,500,clean_mofid,outputs.logKH_CO2_cat
...,...,...,...,...,...,...,...
83,NaN,NaN,NaN,NaN,200,chemical_name_y,outputs.CH4DC_cat
84,NaN,NaN,NaN,NaN,10,chemical_name_y,outputs.logKH_CO2_cat
85,NaN,NaN,NaN,NaN,50,clean_mofid,outputs.logKH_CO2_cat
86,NaN,NaN,NaN,NaN,1000,clean_mofid,outputs.logKH_CO2_cat
